# Build Agent-Powered Workflows Using Snowflake Cortex AI and Managed MCP Servers

This notebook demonstrates how to build a LangGraph agent using MCP tools, and how to use AI observability to evaluate and improve the tool descriptions.

## Overview

We'll build a health research agent that uses MCP tools to query:
- PubMed for medical literature
- Clinical trials databases for trial information

TruLens will automatically trace all tool calls, showing:
- Which MCP tools are being called
- Input arguments and outputs
- Execution time and errors
- Full conversation flow in the dashboard

## Setup

First, start by updating your account settings to create an allow-all network policy, and enable cross-region inference for calling `claude-sonnet-4-5`. You can do this by copying and running [`alter_account_settings.sql`](./alter_account_settings.sql) in a Snowflake SQL worksheet.

## Install python packages in your environment

install the following libraries:

- trulens-connectors-snowflake
- trulens-core
- trulens-providers-cortex
- trulens-apps-langgraph
- langchain-openai OR langchain-snowflake
- langchain-mcp-adapters

🚨🚨 If you are using `langchain-snowflake` for calling LLMs, you will need to use python versions <3.12, >=3.9. Please make sure your python environment is using a supported python version, otherwise `langchain-snowflake` will not be able to be installed. 🚨🚨

In [ ]:
!pip install trulens-connectors-snowflake trulens-core trulens-providers-cortex trulens-apps-langgraph 'langchain-openai==0.3.30' langchain-mcp-adapters 'langchain-snowflake==0.2.2' -q

## Get AI-ready data

In this step, you will get access to AI-ready data from ClinicalTrials.gov and Pubmed Central via [Cortex Knowledge Extensions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-knowledge-extensions/cke-overview), prre-built Cortex Search Services available on the Snowflake marketplace.

1. Open the marketplace listing for the [Clinical Trials Research Database](https://app.snowflake.com/marketplace/listing/GZSTZ67BY9ORD/snowflake-clinical-trials-research-database)

2. Select Get to access the listing

3. Add the public role to grant access to the database created from the listing.

4. Select Get.

5. In the confirmation dialog that appears, select Open to open a Snowsight worksheet with an example query in a new tab, or select Done.

6. Repeat steps 1-5 with the listing for the [PubMed Biomedical Research Corpus](https://app.snowflake.com/marketplace/listing/GZSTZ67BY9OQW/snowflake-pubmed-biomedical-research-corpus?)

## Create MCP Server in Snowflake

Copy [`create_mcp_server.sql`](./create_mcp_server.sql) to a Snowflake worksheet and run. This will create an MCP server with the two Cortex Knowledge Extensions created in the previous step.

<div style="background-color:#ffe6e6; padding:20px; border-width:4px; border-color:#e07070; border-style:double; border-radius:8px; color:#a00000; font-weight:bold;">

## 🔑 API Key Configuration: OpenAI (Optional)
    
🚨 **CRITICAL NOTE:** If you'd like to run the agent with **only Snowflake Cortex LLMs**, you can leave the `OPENAI_API_KEY` commented out. You will choose the Snowflake Cortex model option later in the notebook.
    
</div>

## Add your Snowflake (and optionally OpenAI) credentials below.

To retrieve your Snowflake credentials, follolw the steps below:

1. Get your Programmatic Access Token (PAT) [here](https://app.snowflake.com/_deeplink/settings/authentication)
2. Click on your user profile in the bottom left
3. Choose Connect a tool to Snowflake
4. Copy the value for your Account identifier and paste it in the placeholders below both in SNOWFLAKE_ACCOUNT and in place of <account-id> in the SNOWFLAKE_MCP_SERVER_URL.
5. Copy the value for Login name and paste it in the SNOWFLAKE_USER placeholder.

If you choose to use OpenAI, you can create or retrieve an API key [here](https://platform.openai.com/api-keys).

In [ ]:
import os
import logging

logging.getLogger().setLevel(logging.CRITICAL)

# Configure API keys and MCP server connection
# os.environ["OPENAI_API_KEY"] = "sk-proj-..."
os.environ["SNOWFLAKE_PAT"] = "ey..."
os.environ["SNOWFLAKE_ACCOUNT"] = "..."
os.environ["SNOWFLAKE_USER"] = "..."
os.environ["SNOWFLAKE_MCP_SERVER_URL"] = "https://<account-id>.snowflakecomputing.com/api/v2/databases/HEALTH_DB/schemas/PUBLIC/mcp-servers/HEALTH_MCP_SERVER"

In [ ]:
from snowflake.snowpark import Session
import os

snowflake_connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PAT"),
    "warehouse": "MCP_WH",
    "database": "HEALTH_DB",
    "schema": "PUBLIC",
}

snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

## Create MCP Client and Get Tools

We'll use the `MultiServerMCPClient` from `langchain_mcp_adapters` to connect to the health research MCP server and retrieve available tools.


In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import START
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

client = MultiServerMCPClient({
    "health_research": {
        "transport": "streamable_http",
        "url": os.environ["SNOWFLAKE_MCP_SERVER_URL"],
        "headers": {"Authorization": f"Bearer {os.environ['SNOWFLAKE_PAT']}",
        },
    }
})
tools = await client.get_tools()

<div style="background-color:#e6f7ff; padding:15px; border-width:3px; border-color:#91d5ff; border-style:solid; border-radius:6px; **color:#003a61;**">
    
### 🟢 **Option 1: OpenAI via LangChain**
    
**Action:** **UNCOMMENT and RUN** the code cell below if you are using LangChain with an **OpenAI API Key** (`gpt-4o`).
    
</div>

In [ ]:
# from langchain_openai import ChatOpenAI

# # Initialize the LLM
# model = ChatOpenAI(model="gpt-4o")

<div style="background-color:#f0f8ff; padding:15px; border-width:3px; border-color:#adc6ff; border-style:solid; border-radius:6px;">

### 🔵 **Option 2: Snowflake Cortex via LangChain**

**Action:** **UNCOMMENT and RUN** the code cell below if you are using LangChain with **Snowflake Cortex** (`claude-4-sonnet`).

</div>

In [ ]:
from langchain_snowflake import ChatSnowflake

# Initialize chat model
model = ChatSnowflake(
    session=snowpark_session, 
    model="claude-sonnet-4-5", 
    temperature=0.1, 
    max_tokens=1000
)

## Build the LangGraph Agent

Now we'll create a LangGraph application with:
1. **call_model** node - The LLM that decides which tools to use
2. **tools** node - Executes the selected MCP tools
3. **tools_condition** - Routes between the model and tools

The graph will loop between the model and tools until the agent has enough information to answer the question.


In [ ]:
import asyncio

# Define the call_model function
async def call_model(state: MessagesState):
    response = await model.bind_tools(tools).ainvoke(state["messages"])
    return {"messages": response}


# Create the StateGraph
builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_node(ToolNode(tools))
builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools", "call_model")
graph = builder.compile()

class Agent:
    def __init__(self, graph):
        import nest_asyncio
        nest_asyncio.apply()
        self.graph = graph

    async def ainvoke(self, messages):
        """Async version"""
        return await self.graph.ainvoke({"messages": messages})
    
    def invoke(self, messages):
        """Sync wrapper around async method"""
        return asyncio.run(self.ainvoke(messages))

agent = Agent(graph)

In [ ]:
from IPython.display import Image
Image(agent.graph.get_graph().draw_mermaid_png())

In [ ]:
response = agent.invoke("What is the primary indicator for the drug Xeljanz?")

print(response["messages"][-1].content)

## Initialize TruLens Session

Set up TruLens to store traces and evaluations in Snowflake.

In [ ]:
from snowflake.snowpark import Session
from trulens.connectors.snowflake import SnowflakeConnector

sf_connector = SnowflakeConnector(snowpark_session=snowpark_session)

## Create Tool Selection Evaluations

In [ ]:
from trulens.core.feedback.custom_metric import MetricConfig
from trulens.core.feedback.selector import Selector
from trulens.providers.cortex import Cortex

provider = Cortex(
    model_engine="claude-sonnet-4-5", snowpark_session=snowpark_session
)

f_tool_selection = MetricConfig(
    metric_name = "Tool Selection",
    metric_implementation = provider.tool_selection_with_cot_reasons,
    selectors={
        "trace": Selector(trace_level=True),
    },
)

f_tool_calling = MetricConfig(
    metric_name = "Tool Calling",
    metric_implementation = provider.tool_calling_with_cot_reasons,
    selectors={
        "trace": Selector(trace_level=True),
    },
)

metrics_to_compute = [
    f_tool_selection,
    f_tool_calling,
]

## Record Agent Execution with TruLens

Wrap the LangGraph application with `TruGraph` to automatically instrument and trace all executions.

TruLens will capture:
- Each node execution in the graph
- MCP tool calls with their names (e.g., `pubmed_search`, `clinical_trials_search`)
- Input/output states at each step
- LLM generation calls
- Tool routing decisions

The trace will show the complete flow of the agent's reasoning and tool usage.


In [ ]:
from trulens.apps.langgraph import TruGraph

tru_app = TruGraph(
    app=agent,
    app_name="healthcare_research_assistant",
    app_version="base",
    main_method=agent.invoke,
    connector=sf_connector,
)

In [ ]:
import pandas as pd

queries = ["How do semaglutide and tirzepatide compare in published studies, and what head-to-head clinical trials are recruiting patients?",
"What are the latest clinical trials for Alzheimer's disease?",
"What is the primary indicator for the drug Xeljanz?"]

queries_df = pd.DataFrame(queries, columns=["query"])

## Configure Batched Run

Set up a **batched run** configuration of our dataset. This enables:
- Execution of multiple agent invocations
- Organized tracking of related evaluations
- Dataset management for reproducible experiments

In [ ]:
import uuid

from trulens.core.run import Run
from trulens.core.run import RunConfig

run_name = f"health_queries_run_{uuid.uuid4()}"

run_config = RunConfig(
    run_name=run_name,
    dataset_name="health_research_queries",
    source_type="DATAFRAME",
    dataset_spec={"RECORD_ROOT.INPUT": "query"},
)

run: Run = tru_app.add_run(run_config=run_config)


## Execute Batched Run

Start the batched run to invoke the agent on all queries in the dataset. Each invocation will be traced and stored in Snowflake.

In [ ]:
run.start(input_df=queries_df)

## Compute Metrics

In [ ]:
import time

while run.get_status() != "INVOCATION_COMPLETED":
    print(f"Status: {run.get_status()}")
    time.sleep(3)

print("Invocations complete, computing metrics...")
run.compute_metrics(metrics_to_compute)

In [ ]:
run.get_status()

---

# Agent V2: Improved Tool Descriptions

The base agent had two critical issues identified through TruLens evaluation:

## Problems with Base Version

**1. Poor Tool Selection (Score: Low)**
- Agent didn't use tools for factual medical queries (e.g., "What is Xeljanz indicated for?")
- Relied on internal knowledge instead of authoritative sources
- Missed opportunities to verify medical information

**2. Tool Calling Errors (Multiple Failures)**
- Agent repeatedly tried to specify columns (`title`, `abstract`, `brief_description`, `disease_stage`)
- These columns aren't indexed in Cortex Search, causing errors
- No adaptation after errors - kept making the same mistakes
- Example errors: `"Column abstract was not indexed in this Cortex Search Service"`

## V2 Solution: Enhanced Tool Descriptions

The V2 MCP server (`health_mcp_server_v2`) fixes these issues with improved tool descriptions:

### Fix 1: Mandate Tool Usage
- Tools marked as **"AUTHORITATIVE"** sources
- Explicit **"MANDATORY USE for"** statements for specific query types
- Lists exact use cases: drug indications, efficacy data, safety profiles, etc.

### Fix 2: Prevent Column Specification Errors
- Clear instruction: **"Only provide 'query' parameter"**
- Agent will not specify columns to search that are not available

### Expected Improvements
- ✅ Higher Tool Selection scores (agent uses tools consistently)
- ✅ Higher Tool Calling scores (no more column errors)

## Setup Required

Before running V2 cells, create the improved MCP server:
```sql
-- Run create_mcp_server_v2.sql in Snowflake
```


## Implementing Agent V2

Now let's build and test the V2 agent with improved tool descriptions.

**Prerequisites:** Ensure you've run `create_mcp_server_v2.sql` in Snowflake to create the `health_mcp_server_v2` MCP server.

## Create V2 Client with Improved MCP Server

In [ ]:
# Connect to the V2 MCP server
os.environ["SNOWFLAKE_MCP_SERVER_V2_URL"] = os.environ["SNOWFLAKE_MCP_SERVER_URL"].replace(
    "health_mcp_server", "health_mcp_server_v2"
)

client_v2 = MultiServerMCPClient({
    "health_research_v2": {
        "transport": "streamable_http",
        "url": os.environ["SNOWFLAKE_MCP_SERVER_V2_URL"],
        "headers": {"Authorization": f"Bearer {os.environ['SNOWFLAKE_PAT']}"},
    }
})
tools_v2 = await client_v2.get_tools()

## Build Agent V2

In [ ]:
# Build model with V2 tools
model_v2 = ChatOpenAI(model="gpt-4o")

async def call_model_v2(state: MessagesState):
    response = await model_v2.bind_tools(tools_v2).ainvoke(state["messages"])
    return {"messages": response}

builder_v2 = StateGraph(MessagesState)
builder_v2.add_node(call_model_v2)
builder_v2.add_node(ToolNode(tools_v2))
builder_v2.add_edge(START, "call_model_v2")
builder_v2.add_conditional_edges("call_model_v2", tools_condition)
builder_v2.add_edge("tools", "call_model_v2")
graph_v2 = builder_v2.compile()

agent_v2 = Agent(graph_v2)

## Wrap V2 Agent with TruGraph

In [ ]:
tru_app_v2 = TruGraph(
    app=agent_v2,
    app_name="healthcare_research_assistant-build-hol",
    app_version="v2_improved_descriptions",
    main_method=agent_v2.invoke,
    connector=sf_connector,
)

## Run V2 Agent on Same Queries

In [ ]:
run_name_v2 = f"health_queries_run_v2_{uuid.uuid4()}"

run_config_v2 = RunConfig(
    run_name=run_name_v2,
    dataset_name="health_research_queries",
    source_type="DATAFRAME",
    dataset_spec={"RECORD_ROOT.INPUT": "query"},
)

run_v2: Run = tru_app_v2.add_run(run_config=run_config_v2)

In [ ]:
run_v2.start(input_df=queries_df)

## Compute Metrics for V2

In [ ]:
import time

In [ ]:
while run_v2.get_status() != "INVOCATION_COMPLETED":
    print(f"V2 Status: {run_v2.get_status()}")
    time.sleep(3)

print("V2 invocations complete, computing metrics...")
run_v2.compute_metrics(metrics_to_compute)

## Compare results

By visiting the Comparison tab for Health Research Agent, we can compare how the agent performs with both versions of our MCP Server (before and after the tool description change).

![agent_version_comparison.png](agent_version_comparison.png)

By updating the tool descritions to mandate tool usage for specified use cases and specifying the search parameter, our agent is able to resolve the tool selection and tool calling failures from the first version of the agent.